# BASIC CRUD OPERATIONS

In [2]:
import pandas as pd
from datetime import datetime
from pymongo import MongoClient
# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['eduhub_db']

In [3]:
print("Collections in eduhub:")
print(db.list_collection_names())

Collections in eduhub:
['enrollments', 'assignments', 'submissions', 'users', 'lessons', 'courses']


## CREATE OPERATIONS

### Defining Necassary Functions

In [4]:
#Function for adding users/
def add_user(db, user_data):
    try:
        db.users.insert_one(user_data)
        print(f"Added user: {user_data.get('userId')}")
    except Exception as e:
        print(f"Error adding user {user_data.get('userId')}: {e}")


#Function for adding course
def add_course(db, course_data):
    try:
        db.courses.insert_one(course_data)
        print(f"Added course: {course_data.get('courseId')}")
    except Exception as e:
        print(f"Error adding course {course_data.get('courseId')}: {e}")


#Functions for adding Enrollment
def add_enrollment(db, enrollment_data):
    try:
        db.enrollments.insert_one(enrollment_data)
        print(f"Enrolled student {enrollment_data.get('studentId')} in course {enrollment_data.get('courseId')}")
    except Exception as e:
        print(f"Error enrolling student {enrollment_data.get('studentId')}: {e}")


#Function for addind lesson
def add_lesson(db, lesson_data):
    try:
        db.lessons.insert_one(lesson_data)
        print(f"Added lesson {lesson_data.get('lessonId')} to course {lesson_data.get('courseId')}")
    except Exception as e:
        print(f"Error adding lesson {lesson_data.get('lessonId')}: {e}")

In [5]:
# --- Adding a new student ---
new_student = {
    "userId": "STU020",
    "email": "student17@example.com",
    "firstName": "Ada",
    "lastName": "Eze",
    "role": "student",
    "dateJoined": datetime.now(),
    "profile": {
        "bio": "Passionate learner exploring data science and analytics.",
        "avatar": "https://example.com/avatar16.png",
        "skills": ["Python", "SQL"]
    },
    "isActive": True
}
add_user(db, new_student)


# --- Adding a new course ---
new_course = {
    "courseId": "CRS012",
    "title": "Data Visualization with Power BI",
    "description": "Learn to design and develop interactive dashboards with Power BI.",
    "instructorId": "INST001",
    "category": "Data Science",
    "level": "beginner",
    "duration": 40,
    "price": 99.99,
    "tags": ["data", "visualization", "business-intelligence"],
    "createdAt": datetime.now(),
    "updatedAt": datetime.now(),
    "isPublished": True
}
add_course(db, new_course)

# --- Enrolling a student in a course ---
new_enrollment = {
    "enrollmentId": "ENR019",
    "studentId": "STU016",
    "courseId": "CRS009",
    "enrolledAt": datetime.now(),
    "status": "active"
}
add_enrollment(db, new_enrollment)

# --- Adding a new lesson to an existing course ---
new_lesson = {
    "lessonId": "LES030",
    "courseId": "CRS009",
    "title": "Getting Started with Power BI",
    "content": "This lesson introduces the Power BI interface and its core features.",
    "order": 1,
    "resources": ["https://docs.microsoft.com/en-us/power-bi/"],
    "createdAt": datetime.now(),
    "updatedAt": datetime.now()
}
add_lesson(db, new_lesson)

Added user: STU020
Added course: CRS012
Enrolled student STU016 in course CRS009
Added lesson LES030 to course CRS009


## READ OPERATIONS


### Active Students 

In [7]:
#Finding all active student
def get_active_students(db):
    # Fetch all active students and return as a DataFrame
    try:
        active_students = db.users.find(
            {"role": "student", "isActive": True},
            {"_id": 0, "userId": 1, "firstName": 1, "lastName": 1, "email": 1}
        )
        active_students_list = list(active_students)

        df_active_students = pd.DataFrame(active_students_list)

        if not df_active_students.empty:
            df_active_students = df_active_students[['userId', 'firstName', 'lastName', 'email']]
            print("Active Students:\n")
            return df_active_students
        else:
            print("No active students found.")
            return pd.DataFrame()

    except Exception as e:
        print("Error fetching active students:", e)
        return pd.DataFrame()
    
#Use Case 
get_active_students(db)

Active Students:



,userId,firstName,lastName,email
0,STU001,Thomas,Hammond,floresmichael@example.net
1,STU006,Crystal,Hudson,mlawrence@example.org
2,STU007,Brian,Hudson,rodriguezmichael@example.com
3,STU008,Kenneth,Juarez,kyle36@example.com
4,STU009,Brandon,Dean,larry46@example.com
5,STU010,Michelle,Morales,rortiz@example.org
6,STU011,Daniel,Myers,utownsend@example.net
7,STU012,Kelly,Marquez,smithelizabeth@example.net
8,STU015,Donald,Baker,amanda21@example.org
9,STU020,Ada,Eze,student17@example.com


### Retrieving Course Details with Instructor Information

In [10]:
# Retrieving the Course Details 
def get_courses_with_instructor(db):
    # Fetch courses along with instructor information
    try:
        course_with_instructor = db.courses.aggregate([
            {
                "$lookup": {
                    "from": "users",
                    "localField": "instructorId",
                    "foreignField": "userId",
                    "as": "instructor"
                }
            },
            {"$unwind": "$instructor"},
            {
                "$project": {
                    "_id": 0,
                    "courseId": 1,
                    "title": 1,
                    "category": 1,
                    "level": 1,
                    "instructorFirstName": "$instructor.firstName",
                    "instructorLastName": "$instructor.lastName",
                    "instructorEmail": "$instructor.email"
                }
            }
        ])

        courses_list = list(course_with_instructor)

        df_courses = pd.DataFrame(courses_list)

        if not df_courses.empty:
            df_courses = df_courses[['courseId', 'title', 'category', 'level',
                                     'instructorFirstName', 'instructorLastName', 'instructorEmail']]
            
            print("Instructors with courses;\n")
            return df_courses
        else:
            return pd.DataFrame()

    except Exception as e:
        print("Error fetching courses with instructor info:", e)
        return pd.DataFrame()
    
#Use Case 
get_courses_with_instructor(db)

Instructors with courses;



,courseId,title,category,level,instructorFirstName,instructorLastName,instructorEmail
0,CRS001,DevOps Fundamentals,Cloud Computing,intermediate,Matthew,Palmer,abishop@example.com
1,CRS002,Introduction to Data Science,Data Science,intermediate,Matthew,Palmer,abishop@example.com
2,CRS003,Frontend Development with React,Web Development,intermediate,Kristen,Mclean,xcarpenter@example.net
3,CRS004,Serverless Applications on Cloud,Cloud Computing,advanced,Kristen,Mclean,xcarpenter@example.net
4,CRS005,Python for Data Analysis,Data Science,advanced,Mallory,Henson,susanford@example.org
5,CRS006,Introduction to Data Science,Data Science,advanced,Mallory,Henson,susanford@example.org
6,CRS007,Serverless Applications on Cloud,Cloud Computing,beginner,Matthew,Palmer,abishop@example.com
7,CRS008,Introduction to Data Science,Data Science,intermediate,Mallory,Henson,susanford@example.org
8,CRS012,Data Visualization with Power BI,Data Science,beginner,Brian,Richmond,rayamanda@example.net


###  Getting all Courses in a Specific Category

In [11]:
#Getting all courses in a specific category

category = "Data Science"

def get_courses_by_category(db, category):
    # Fetch courses in a given category
    try:
        courses_in_category = db.courses.find(
            {"category": category},
            {"_id": 0, "courseId": 1, "title": 1, "level": 1}
        )
        
        courses_list = list(courses_in_category)
        df_courses_category = pd.DataFrame(courses_list)

        if not df_courses_category.empty:
            df_courses_category = df_courses_category[['courseId', 'title', 'level']]
            print("Courses in specific category:\n")
            return df_courses_category
        else:
            return pd.DataFrame()

    except Exception as e:
        print(f"Error fetching courses in category '{category}': {e}")
        return pd.DataFrame()
    
# Use Case 
get_courses_by_category(db, category)

Courses in specific category:



,courseId,title,level
0,CRS002,Introduction to Data Science,intermediate
1,CRS005,Python for Data Analysis,advanced
2,CRS006,Introduction to Data Science,advanced
3,CRS008,Introduction to Data Science,intermediate
4,CRS012,Data Visualization with Power BI,beginner


### Finding Students Enrolled in a Particular Course


In [15]:
# Finding student enrolled in a particular course

def get_students_in_course(db, course_id):
    try:
        # Aggregate students enrolled in a specific course
        students_in_course = db.enrollments.aggregate([
            { "$match": {"courseId": course_id} },
            {
                "$lookup": {
                    "from": "users",
                    "localField": "studentId",
                    "foreignField": "userId",
                    "as": "student"
                }
            },
            { "$unwind": "$student" },
            {
                "$project": {
                    "_id": 0,
                    "userId": "$student.userId",
                    "firstName": "$student.firstName",
                    "lastName": "$student.lastName",
                    "email": "$student.email"
                }
            }
        ])

        # Convert aggregation cursor to list
        students_list = list(students_in_course)

        # Convert list to DataFrame
        df_students = pd.DataFrame(students_list)

        # Reorder columns
        df_students = df_students[['userId', 'firstName', 'lastName', 'email']]

        # Display the DataFrame
        print(f"Students enrolled in {course_id}:\n")
        

        return df_students

    except Exception as e:
        print(f"Error fetching students for course '{course_id}': {e}")
        return pd.DataFrame()
    
# Use Case 
course_id = "CRS005"
get_students_in_course(db,course_id )

Students enrolled in CRS005:



,userId,firstName,lastName,email
0,STU003,Karen,Parker,usmith@example.com
1,STU012,Kelly,Marquez,smithelizabeth@example.net
2,STU010,Michelle,Morales,rortiz@example.org


### Searching Courses by Title (case-insensitive, partial match)

In [19]:
# Searching courses by Title
search_term = "Data Science"

def search_courses_by_title(db, search_term):
    try:
        # Query courses matching the regex
        matched_courses = db.courses.find(
            {"title": {"$regex": search_term, "$options": "i"}},
            {"_id": 0, "courseId": 1, "title": 1, "category": 1, "level": 1}
        )

        matched_courses_list = list(matched_courses)

        # Convert list to DataFrame
        df_matched_courses = pd.DataFrame(matched_courses_list)

        # Reorder columns
        df_matched_courses = df_matched_courses[['courseId', 'title', 'category', 'level']]

        # Display the DataFrame
        print(f"Courses matching '{search_term}':\n")
        return df_matched_courses

    except Exception as e:
        print(f"Error searching courses with term '{search_term}': {e}")
        return pd.DataFrame()
    
# Use Case 
search_courses_by_title(db, search_term)

Courses matching 'Data Science':



,courseId,title,category,level
0,CRS002,Introduction to Data Science,Data Science,intermediate
1,CRS006,Introduction to Data Science,Data Science,advanced
2,CRS008,Introduction to Data Science,Data Science,intermediate


## UPDATE OPERATIONS

### Update a User’s Profile Information

In [ ]:
#Updating a Useers Profile information
def update_user_profile(db, user_id, bio, avatar, skills):
    try:
        db.users.update_one(
            {"userId": user_id},
            {
                "$set": {
                    "profile.bio": bio,
                    "profile.avatar": avatar,
                    "profile.skills": skills
                }
            }
        )
        print(f"User {user_id} profile updated successfully.")

    except Exception as e:
        print(f"Error updating user {user_id}: {e}")

# Use Case 
update_user_profile(
    db, 
    user_id="STU015", 
    bio="Enthusiastic learner with interest in AI and Machine Learning.",
    avatar="https://example.com/new_avatar.png",
    skills=["Python", "SQL", "Machine Learning"]
)


User STU015 profile updated successfully.


### Mark a Course as Published

In [21]:
# Marking a course as published

def publish_course(db, course_id):
    try:
        db.courses.update_one(
            {"courseId": course_id},
            {"$set": {"isPublished": True, "updatedAt": datetime.now()}}
        )
        print(f"Course {course_id} marked as published successfully.")

    except Exception as e:
        print(f"Error marking course {course_id} as published: {e}")

# Use Case 
publish_course(db, "CRS004")


Course CRS004 marked as published successfully.


### Update Assignment Grades

In [23]:
#Updating Assignment Grades
def update_submission_grade(db, submission_id, grade, feedback, status):
    try:
        db.submissions.update_one(
            {"submissionId": submission_id},
            {
                "$set": {
                    "grade": grade,
                    "feedback": feedback,
                    "status": status,
                    "gradedAt": datetime.now()
                }
            }
        )
        print(f"Updated grade and status for submission {submission_id} successfully.")

    except Exception as e:
        print(f"Error updating grade for submission {submission_id}: {e}")

# Use Case 
update_submission_grade(db, "SUB011", 85, "Great improvement!", "graded")

Updated grade and status for submission SUB011 successfully.


### Add Tags to an Existing Course

In [ ]:
#Adding Tags to an Existing Course
def update_course_tags(db, course_id, tags):
    try:
        db.courses.update_one(
            {"courseId": course_id},
            {"$addToSet": {"tags": {"$each": tags}}}
        )
        print(f"Tags updated for course {course_id} successfully.")

    except Exception as e:
        print(f"Error updating tags for course {course_id}: {e}")

# Use Case
update_course_tags(db, "CRS005", ["AI", "Data Science", "Python"])

Tags updated for course CRS005 successfully.


## DELETE OPERATIONS

### Remove a User (soft delete by setting isActive to false)


In [25]:
# Removing user
def deactivate_user(db, user_id):
    try:
        db.users.update_one(
            {"userId": user_id},
            {"$set": {"isActive": False, "updatedAt": datetime.now()}}
        )
        print(f"User {user_id} has been deactivated successfully.")
    except Exception as e:
        print(f"Error deactivating user {user_id}: {e}")

# Use Case 
deactivate_user(db, "STU006")

User STU006 has been deactivated successfully.


### Deleting  an Enrollment

In [26]:
#Deleting an Enrollment
def delete_enrollment(db, enrollment_id):
    try:
        delete_result = db.enrollments.delete_one({"enrollmentId": enrollment_id})
        print(f"Deleted {delete_result.deleted_count} enrollment(s) with ID {enrollment_id}")
    except Exception as e:
        print(f"Error deleting enrollment {enrollment_id}: {e}")

# Use Case 
delete_enrollment(db, "ENR005")

Deleted 1 enrollment(s) with ID ENR005


###  Remove a Lesson from a Course

In [27]:
# Removing a lesson from a Course
def delete_lesson(db, lesson_id):
    try:
        delete_result = db.lessons.delete_one({"lessonId": lesson_id})
        print(f"Deleted {delete_result.deleted_count} lesson(s) with ID {lesson_id}")
    except Exception as e:
        print(f"Error deleting lesson {lesson_id}: {e}")

# Use Case 
delete_lesson(db, "LES008")


Deleted 1 lesson(s) with ID LES008
